In [1]:
import clip
import os
from torch import nn
import numpy as np
import torch
import torch.nn.functional as nnf
import sys
from tqdm import tqdm, trange
import PIL.Image
from IPython.display import Image
import pandas as pd


device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
df_train = pd.read_csv('new_train.csv')
df_train.question = df_train.trans
# df_train.answer = df_train.trans_ans

In [4]:
def read_video(path, size=336, num_frames=12):
    vidcap = cv2.VideoCapture(path)
    video_len = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    frames = []
    
    for i in range(video_len % num_frames, video_len, video_len // num_frames):
        k = i
        success, frame = vidcap.read(k)
        
        while not success:
            k += 1
            success, frame = vidcap.read(k)
            
        frames.append(Image.fromarray(cv2.resize(frame, dsize=(size, size), interpolation=cv2.INTER_CUBIC)))

    return frames

In [5]:
device = torch.device('cuda')
clip_model_type = "ViT-L/14@336px"

video_path =  'videos'

clip_model, preprocess = clip.load(clip_model_type, device=device, jit=False)

In [6]:
clip_model.to(device)

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
    (ln_pre): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)
          )
          (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=1024, out_features=4096, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=4096, out_features=1024, bias=True)
          )
          (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=1024, out_features=1024, bias=True)


In [15]:
import gc
import io


import random
import numpy as np

import torchvision
import transformers
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import pandas as pd
from torch.utils.data import Dataset
from tqdm import tqdm
from dataclasses import dataclass, field
import torchvision.transforms as T
import torchvision.transforms.functional as TF
import cv2
from PIL import Image
import pickle
import sys
from tqdm.contrib import tzip
from tqdm import tqdm

import torch
import torch.nn as nn
from torch.nn import functional as nnf
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, AdamW, GPT2Tokenizer, GPT2LMHeadModel, get_linear_schedule_with_warmup

import os
import pickle
import sys
import argparse
import json
from typing import Tuple, Optional, Union
from torch.cuda.amp import autocast

from transformers.optimization import Adafactor, AdafactorSchedule
import torch

from torch.utils.checkpoint import checkpoint_sequential

class ClipCocoDataset(Dataset): 
    def __init__(self, data_path: str,  prefix_length=10, gpt2_type = "sberbank-ai/rugpt3small_based_on_gpt2",
                 normalize_prefix=False):
        self.tokenizer = AutoTokenizer.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')
        self.prefix_length = prefix_length
        self.normalize_prefix = normalize_prefix
        with open(data_path, 'rb') as f:
            all_data = pickle.load(f)
        print("Data size is %0d" % len(all_data["clip_embedding"]))
        sys.stdout.flush()
        self.prefixes = all_data["clip_embedding"]
        answers_raw = all_data["answers"]
        print(answers_raw[:30])
        
        self.answers = answers_raw
        
        self.answers_tokens = []
        self.answer2embedding = []
        max_seq_len = 0
        i=0
        for answer in tqdm(answers_raw):
                self.answers_tokens.append(torch.tensor(self.tokenizer.encode(answer), dtype=torch.int64))
                self.answer2embedding.append(self.prefixes[i])
                i+=1
                max_seq_len = max(max_seq_len, self.answers_tokens[-1].shape[0])
    
        all_len = torch.tensor([len(self.answers_tokens[i]) for i in range(len(self))]).float()
        self.max_seq_len = min(int(all_len.mean() + all_len.std() * 10), int(all_len.max()))

    def pad_tokens(self, item: int):
        tokens = self.answers_tokens[item]
        padding = self.max_seq_len - tokens.shape[0]
        if padding > 0:
            tokens = torch.cat((tokens, torch.zeros(padding, dtype=torch.int64) - 1))
            self.answers_tokens[item] = tokens
        elif padding < 0:
            tokens = tokens[:self.max_seq_len]
            self.answers_tokens[item] = tokens
        mask = tokens.ge(0)  # mask is zero where we out of sequence
        tokens[~mask] = 0
        mask = mask.float()
        mask = torch.cat((torch.ones(self.prefix_length), mask), dim=0)  # adding prefix mask
        return tokens, mask
    
    def __len__(self) -> int:
        return len(self.answers_tokens)

    def __getitem__(self, item):
        tokens, mask = self.pad_tokens(item)
        prefix = self.prefixes[item]
        if self.normalize_prefix:
            prefix = prefix.float()
            prefix = prefix / prefix.norm(2, -1)
        return tokens, mask, prefix

In [16]:
dataset = ClipCocoDataset('Features_train_2.pkl', prefix_length=50, normalize_prefix=False)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Data size is 18993
['Question: человек в черных штанах мальчик или девочка? Answer: мужской', 'Question: человек в черных штанах мальчик или девочка? Answer: мальчик', 'Question: человек в черных штанах мальчик или девочка? Answer: мужской', 'Question: Барабанщик носит одежду с длинными или короткими рукавами?? Answer: длинный', 'Question: Барабанщик носит одежду с длинными или короткими рукавами?? Answer: длинный', 'Question: это день или ночь на видео? Answer: день', 'Question: это день или ночь на видео? Answer: день', 'Question: человек на видео носит длинные или короткие рукава? Answer: длинный', 'Question: сцена ремонта происходит в помещении или на открытом воздухе? Answer: открытый', 'Question: сцена ремонта происходит в помещении или на открытом воздухе? Answer: открытый', 'Question: сцена катания на лодке происходит в помещении или на открытом воздухе? Answer: открытый', 'Question: насколько опасен каякинг? Answer: обычно', 'Question: насколько сложно полировать ногти? Answer

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 18993/18993 [00:01<00:00, 16689.81it/s]


In [17]:
import wandb
wandb.init(project="clip_caption_video_base")


class MLP(nn.Module):
    def __init__(self, sizes: Tuple[int, ...], bias=True, act=nn.Tanh):
        super(MLP, self).__init__()
        layers = []
        for i in range(len(sizes) - 1):
            layers.append(nn.Linear(sizes[i], sizes[i + 1], bias=bias))
            if i < len(sizes) - 2:
                layers.append(act())
        self.model = nn.Sequential(*layers)
    
    #@autocast()  
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)

def freeze(
    model,
    freeze_emb=False,
    freeze_ln=False,
    freeze_attn=True,
    freeze_ff=True,
    freeze_other=True,
):

    for name, p in model.named_parameters():
        name = name.lower()
        if 'ln' in name or 'norm' in name:
            p.requires_grad = not freeze_ln
        elif 'embeddings' in name:
            p.requires_grad = not freeze_emb
        elif 'mlp' in name:
            p.requires_grad = not freeze_ff
        elif 'attn' in name:
            p.requires_grad = not freeze_attn
        else:
            p.requires_grad = not freeze_other
           
    return model

class ClipCaptionModel(nn.Module):
    def __init__(self, backbone=None, prefix_length: int = 50, prefix_size: int = 768):
          super(ClipCaptionModel, self).__init__()
          self.prefix_length = prefix_length
        
          self.gpt = AutoModelForCausalLM.from_pretrained(backbone)
          #self.gpt = freeze(self.gpt)
          self.gpt_embedding_size = self.gpt.transformer.wte.weight.shape[1]
          self.clip_project = MLP((prefix_size, (self.gpt_embedding_size * prefix_length) // 2,
                                  self.gpt_embedding_size * prefix_length))

    def get_dummy_token(self, batch_size: int, device: torch.device) -> torch.Tensor:
        return torch.zeros(batch_size, self.prefix_length, dtype=torch.int64, device=device)
    
    # @autocast() 
    def forward(self, tokens: torch.Tensor, prefix: torch.Tensor, mask: Optional[torch.Tensor] = None,
                labels: Optional[torch.Tensor] = None):
        
        embedding_text = self.gpt.transformer.wte(tokens)
        prefix_projections = self.clip_project(prefix).view(-1, self.prefix_length, self.gpt_embedding_size)

        embedding_cat = torch.cat((prefix_projections, embedding_text), dim=1)
        if labels is not None:
            dummy_token = self.get_dummy_token(tokens.shape[0], tokens.device)
            labels = torch.cat((dummy_token, tokens), dim=1)
        out = self.gpt(inputs_embeds=embedding_cat, labels=labels, attention_mask=mask)
        return out

class ClipCaptionPrefix(ClipCaptionModel):
    def parameters(self, recurse: bool = True):
        return self.clip_project.parameters()

    def train(self, mode: bool = True):
        super(ClipCaptionPrefix, self).train(mode)
        self.gpt.eval()
        return self

def train(dataset: ClipCocoDataset, model: ClipCaptionModel, args,
          warmup_steps: int = 5000, output_dir: str = ".", output_prefix: str = ""):

    device = torch.device('cuda')

    batch_size = args.bs
    epochs = args.epochs
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    model = model.to(device)
    model = freeze(model)
    model.train()
    optimizer = AdamW(model.parameters(),lr=args.lr,betas=(0.9, 0.995))

    train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)

    for epoch in range(epochs):
        print(f">>> Training epoch {epoch}")
        sys.stdout.flush()
        progress = tqdm(total=len(train_dataloader), desc=output_prefix)
        step=0
        for idx, (tokens, mask, prefix) in enumerate(train_dataloader):
            model.zero_grad()
            step += 1
            tokens, mask, prefix = tokens.to(device), mask.to(device), prefix.to(device, dtype=torch.float32)
            
            outputs = model(tokens, prefix, mask)
            logits = outputs.logits[:, dataset.prefix_length - 1: -1]

            loss = nnf.cross_entropy(logits.reshape(-1, logits.shape[-1]), tokens.flatten(), ignore_index=0)

            segments = 2
            
            loss.backward()
            optimizer.step()

            progress.set_postfix({"loss": loss.item()})
            
            wandb.log({"loss":  loss.item()})
            
            progress.update()
            

            del tokens
            del mask
            del prefix
            torch.clear_autocast_cache()
            torch.cuda.empty_cache()
            
            if (idx + 1) % 7000 == 0:
                torch.save(
                    model.state_dict(),
                    
                    os.path.join(output_dir, f"{output_prefix}_latest.pt"),
                )
        progress.close()
        if (epoch + 1) % args.save_every == 0:
            torch.save(
                model.state_dict(),
                f"checkpoints/{output_prefix}-{epoch:03d}.pt",
            )
    return model



class Args():
    def __init__(self):
        self.data = 'Features_train_2.pkl'
        self.out_dir = 'checkpoints'
        self.prefix = 'prefix_1'
        self.epochs = 20
        self.save_every = 1
        self.prefix_length = 50
        self.bs = 16
        self.only_prefix = False
        self.lr = 3e-5
        


def main():
    args = Args()


    prefix_length = args.prefix_length

    dataset = ClipCocoDataset(args.data, prefix_length)
    
    model = ClipCaptionModel(backbone = 'sberbank-ai/rugpt3small_based_on_gpt2', prefix_length = 50)
    print("Train both prefix and GPT")
    sys.stdout.flush()
    train(dataset, model, args, output_dir=args.out_dir, output_prefix=args.prefix)

wandb: Currently logged in as: vladimir_pvorobev (humarin). Use `wandb login --relogin` to force relogin


In [18]:
model = ClipCaptionModel(backbone = 'sberbank-ai/rugpt3small_based_on_gpt2', prefix_length = 20)


with open('checkpoints/prefix_1-009.pkl', 'rb') as r:
    model.load_state_dict(pickle.load(r))

In [19]:
tokenizer = AutoTokenizer.from_pretrained('sberbank-ai/rugpt3small_based_on_gpt2')
tokenizer.save_pretrained('rugpt3small_based_on_gpt2_tok')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


('rugpt3small_based_on_gpt2_tok\\tokenizer_config.json',
 'rugpt3small_based_on_gpt2_tok\\special_tokens_map.json',
 'rugpt3small_based_on_gpt2_tok\\vocab.json',
 'rugpt3small_based_on_gpt2_tok\\merges.txt',
 'rugpt3small_based_on_gpt2_tok\\added_tokens.json',
 'rugpt3small_based_on_gpt2_tok\\tokenizer.json')

In [20]:
model.gpt.save_pretrained('rugpt3medium_based_on_gpt2')

In [21]:
model

ClipCaptionModel(
  (gpt): GPT2LMHeadModel(
    (transformer): GPT2Model(
      (wte): Embedding(50264, 768)
      (wpe): Embedding(2048, 768)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0): GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): GPT2MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
            (act): NewGELUActivation()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (1): GPT2Block(
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
       

In [ ]:
main()